In [5]:
import pandas as pd
import numpy as np
import io
import requests

In [7]:
url = 'https://phl.carto.com/api/v2/sql?q=SELECT+*,+ST_Y(the_geom)+AS+lat,+ST_X(the_geom)+AS+lng+FROM+opa_properties_public&filename=opa_properties_public&format=csv&skipfields=cartodb_id,the_geom,the_geom_webmercator'
df = pd.read_csv(url)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3050: DtypeWarning: Columns (1,3,10,28,29,30,31,32,33,70) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
df

,number_of_rooms,assessment_date,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,...,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,objectid,lat,lng
0,6.0,NaN,"153'7 1/2""N ALLEGHENY",1.77592e+06,O30,ROW 2 STY MASONRY,1,Single Family,521.0,NaN,...,NaN,NaN,I,1935,Y,191342205,RM1,603644216,39.998934,-75.127498
1,6.0,NaN,"16'4 1/2""S WILLARD",1.74793e+06,O30,ROW 2 STY MASONRY,1,Single Family,521.0,NaN,...,NaN,NaN,I,1935,Y,191342205,RM1,603644217,39.999272,-75.127344
2,6.0,NaN,SWC WILLARD,3.50011e+06,O30,ROW 2 STY MASONRY,1,Single Family,521.0,NaN,...,NaN,NaN,I,1935,Y,191342205,RM1,603644218,39.999317,-75.127335
3,6.0,NaN,NWC WILLA RD,0,O30,ROW 2 STY MASONRY,1,Single Family,521.0,NaN,...,NaN,NaN,I,1935,Y,191342227,RM1,603644219,39.999468,-75.127297
4,6.0,NaN,SEC WILLARD,3.15587e+06,O30,ROW 2 STY MASONRY,1,Single Family,520.0,NaN,...,NaN,NaN,I,1935,Y,191342204,RM1,603644220,39.999256,-75.126923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581451,6.0,NaN,270' N OF ERIE AVE,0000000,O30,ROW 2 STY MASONRY,1,Single Family,830.0,N,...,NaN,NaN,I,1940,Y,191403237,RSA5,604223781,40.008865,-75.141112
581452,6.0,NaN,224.5' SE DORCAS,2502246,H11,SEMI/DET 1 STY MAS+OTHER,1,Single Family,700.0,Y,...,NaN,NaN,I,1985,NaN,191522218,RSA3,604223782,40.068118,-75.061519
581453,7.0,NaN,83 080/1000' W 56TH,1413542,K30,S/D W/B GAR 2 STY MASONRY,1,Single Family,710.0,NaN,...,NaN,NaN,I,1925,Y,191311327,RSA3,604223783,39.992299,-75.236619
581454,6.0,NaN,"131'9 3/4"" N OF RIDGE",2713169,O30,ROW 2 STY MASONRY,1,Single Family,151.0,NaN,...,NaN,NaN,I,1915,Y,191211120,RSA5,604223784,39.989151,-75.182882


In [20]:
rename_dict = {
    'number_of_rooms': {
        'delete': 0,
        'new_name': '# ROOMS',
    },
    'assessment_date': {
        'delete': 0,
        'new_name': 'assessment_date',
    },
    'beginning_point': {
        'delete': 1,
    },
    'book_and_page': {
        'delete': 1,
    },
    'building_code': {
        'delete': 1,
    },
    'building_code_description': {
        'delete': 0,
        'new_name': 'BLDG CODE',
    },
    'category_code': {
        'delete': 1,
    },
    'category_code_description': {
        'delete': 0,
        'new_name': 'BLDG CAT',
    },
    'census_tract': {
        'delete': 1,
    },
    'central_air': {
        'delete': 0,
        'new_name': 'CENTRAL AIR',
    },
    ''
}